In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=7a5bd745ecd60eeeb7ddd2ba1959da294521235133455f4fa7ae33b9420f1d5f
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [ ]:
!unzip '/content/drive/MyDrive/fewshot.zip'

Streaming output truncated to the last 5000 lines.
  inflating: fewshot/fss1000/sea_urchin/9.png  
   creating: fewshot/fss1000/sewing_machine/
  inflating: fewshot/fss1000/sewing_machine/1.jpg  
  inflating: fewshot/fss1000/sewing_machine/1.png  
  inflating: fewshot/fss1000/sewing_machine/10.jpg  
  inflating: fewshot/fss1000/sewing_machine/10.png  
  inflating: fewshot/fss1000/sewing_machine/2.jpg  
  inflating: fewshot/fss1000/sewing_machine/2.png  
  inflating: fewshot/fss1000/sewing_machine/3.jpg  
  inflating: fewshot/fss1000/sewing_machine/3.png  
  inflating: fewshot/fss1000/sewing_machine/4.jpg  
  inflating: fewshot/fss1000/sewing_machine/4.png  
  inflating: fewshot/fss1000/sewing_machine/5.jpg  
  inflating: fewshot/fss1000/sewing_machine/5.png  
  inflating: fewshot/fss1000/sewing_machine/6.jpg  
  inflating: fewshot/fss1000/sewing_machine/6.png  
  inflating: fewshot/fss1000/sewing_machine/7.jpg  
  inflating: fewshot/fss1000/sewing_machine/7.png  
  inflating: fewshot/f

In [ ]:
!nvidia-smi

Thu Jul 15 06:28:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip '/content/drive/MyDrive/b1_model.zip'

Archive:  /content/drive/MyDrive/b1_model.zip
   creating: b1_model/
  inflating: b1_model/segmentation_network_99999_1_way_5shot.pkl  
  inflating: b1_model/decoder_network_99999_1_way_5shot.pkl  
  inflating: b1_model/efficientnet_encoder_99999_1_way_5shot.pkl  


In [ ]:
!ls

b1_model  drive  fewshot  fss_test_set.txt  sample_data


In [ ]:
import argparse
import sys
sys.argv=['']
del sys
parser = argparse.ArgumentParser(description="One Shot Visual Recognition")
parser.add_argument("-g","--gpu",type=int, default=0)
args = parser.parse_args()
GPU = args.gpu

In [ ]:
import torch
import torch.nn as nn


def patch_first_conv(model, in_channels):
    """Change first convolution layer input channels.
    In case:
        in_channels == 1 or in_channels == 2 -> reuse original weights
        in_channels > 3 -> make random kaiming normal initialization
    """

    # get first conv
    for module in model.modules():
        if isinstance(module, nn.Conv2d):
            break

    # change input channels for first conv
    module.in_channels = in_channels
    weight = module.weight.detach()
    reset = False

    if in_channels == 1:
        weight = weight.sum(1, keepdim=True)
    elif in_channels == 2:
        weight = weight[:, :2] * (3.0 / 2.0)
    else:
        reset = True
        weight = torch.Tensor(
            module.out_channels,
            module.in_channels // module.groups,
            *module.kernel_size
        )

    module.weight = nn.parameter.Parameter(weight)
    if reset:
        module.reset_parameters()


def replace_strides_with_dilation(module, dilation_rate):
    """Patch Conv2d modules replacing strides with dilation"""
    for mod in module.modules():
        if isinstance(mod, nn.Conv2d):
            mod.stride = (1, 1)
            mod.dilation = (dilation_rate, dilation_rate)
            kh, kw = mod.kernel_size
            mod.padding = ((kh // 2) * dilation_rate, (kh // 2) * dilation_rate)

            # Kostyl for EfficientNet
            if hasattr(mod, "static_padding"):
                mod.static_padding = nn.Identity()


In [ ]:
import torch
import torch.nn as nn
from typing import List
from collections import OrderedDict



class EncoderMixin:
    """Add encoder functionality such as:
        - output channels specification of feature tensors (produced by encoder)
        - patching first convolution for arbitrary input channels
    """

    @property
    def out_channels(self):
        """Return channels dimensions for each tensor of forward output of encoder"""
        return self._out_channels[: self._depth + 1]

    def set_in_channels(self, in_channels):
        """Change first convolution channels"""
        if in_channels == 3:
            return

        self._in_channels = in_channels
        if self._out_channels[0] == 3:
            self._out_channels = tuple([in_channels] + list(self._out_channels)[1:])

        patch_first_conv(model=self, in_channels=in_channels)

    def get_stages(self):
        """Method should be overridden in encoder"""
        raise NotImplementedError

    def make_dilated(self, stage_list, dilation_list):
        stages = self.get_stages()
        for stage_indx, dilation_rate in zip(stage_list, dilation_list):
            replace_strides_with_dilation(
                module=stages[stage_indx],
                dilation_rate=dilation_rate,
            )


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ChannelSELayer(nn.Module):
    """
    Re-implementation of Squeeze-and-Excitation (SE) block described in:
        *Hu et al., Squeeze-and-Excitation Networks, arXiv:1709.01507*
    """

    def __init__(self, num_channels, reduction_ratio=2):
        """
        :param num_channels: No of input channels
        :param reduction_ratio: By how much should the num_channels should be reduced
        """
        super(ChannelSELayer, self).__init__()
        num_channels_reduced = num_channels // reduction_ratio
        self.reduction_ratio = reduction_ratio
        self.fc1 = nn.Linear(num_channels, num_channels_reduced, bias=True)
        self.fc2 = nn.Linear(num_channels_reduced, num_channels, bias=True)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):
        """
        :param input_tensor: X, shape = (batch_size, num_channels, H, W)
        :return: output tensor
        """
        batch_size, num_channels, H, W = input_tensor.size()
        # Average along each channel
        squeeze_tensor = input_tensor.view(batch_size, num_channels, -1).mean(dim=2)

        # channel excitation
        fc_out_1 = self.relu(self.fc1(squeeze_tensor))
        fc_out_2 = self.sigmoid(self.fc2(fc_out_1))

        a, b = squeeze_tensor.size()
        output_tensor = torch.mul(input_tensor, fc_out_2.view(a, b, 1, 1))
        return output_tensor


class SpatialSELayer(nn.Module):
    """
    Re-implementation of SE block -- squeezing spatially and exciting channel-wise described in:
        *Roy et al., Concurrent Spatial and Channel Squeeze & Excitation in Fully Convolutional Networks, MICCAI 2018*
    """

    def __init__(self, num_channels):
        """
        :param num_channels: No of input channels
        """
        super(SpatialSELayer, self).__init__()
        self.conv = nn.Conv2d(num_channels, 1, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor, weights=None):
        """
        :param weights: weights for few shot learning
        :param input_tensor: X, shape = (batch_size, num_channels, H, W)
        :return: output_tensor
        """
        # spatial squeeze
        batch_size, channel, a, b = input_tensor.size()

        if weights is not None:
            weights = torch.mean(weights, dim=0)
            weights = weights.view(1, channel, 1, 1)
            out = F.conv2d(input_tensor, weights)
        else:
            out = self.conv(input_tensor)
        squeeze_tensor = self.sigmoid(out)

        # spatial excitation
        # print(input_tensor.size(), squeeze_tensor.size())
        squeeze_tensor = squeeze_tensor.view(batch_size, 1, a, b)
        output_tensor = torch.mul(input_tensor, squeeze_tensor)
        #output_tensor = torch.mul(input_tensor, squeeze_tensor)
        return output_tensor


class ChannelSpatialSELayer(nn.Module):
    """
    Re-implementation of concurrent spatial and channel squeeze & excitation:
        *Roy et al., Concurrent Spatial and Channel Squeeze & Excitation in Fully Convolutional Networks, MICCAI 2018, arXiv:1803.02579*
    """

    def __init__(self, num_channels, reduction_ratio=2):
        """
        :param num_channels: No of input channels
        :param reduction_ratio: By how much should the num_channels should be reduced
        """
        super(ChannelSpatialSELayer, self).__init__()
        self.cSE = ChannelSELayer(num_channels, reduction_ratio).cuda(GPU)
        self.sSE = SpatialSELayer(num_channels).cuda(GPU)

    def forward(self, input_tensor):
        """
        :param input_tensor: X, shape = (batch_size, num_channels, H, W)
        :return: output_tensor
        """
        output_tensor = torch.max(self.cSE(input_tensor), self.sSE(input_tensor))
        return output_tensor


In [ ]:
import torch.nn as nn
from efficientnet_pytorch import EfficientNet, model
from efficientnet_pytorch.utils import url_map, url_map_advprop, get_model_params



class EfficientNetEncoder(EfficientNet, EncoderMixin):
    def __init__(self, stage_idxs, out_channels, model_name, depth=5):

        blocks_args, global_params = get_model_params(model_name, override_params=None)
        super().__init__(blocks_args, global_params)

        self._stage_idxs = stage_idxs
        self._out_channels = out_channels
        self._depth = depth
        self._in_channels = 3
        #self.scse = ChannelSpatialSELayer(self._in_channels)
       

        del self._fc

    def get_stages(self):
        return [
            nn.Identity(),
            nn.Sequential(self._conv_stem, self._bn0, self._swish),
            self._blocks[:self._stage_idxs[0]],
            self._blocks[self._stage_idxs[0]:self._stage_idxs[1]],
            self._blocks[self._stage_idxs[1]:self._stage_idxs[2]],
            self._blocks[self._stage_idxs[2]:],
        ]

    def forward(self, x):
        stages = self.get_stages()
        #print(stages)

        block_number = 0.
        drop_connect_rate = self._global_params.drop_connect_rate
        #print(drop_connect_rate)

        features = []
        for i in range(self._depth + 1):

            # Identity and Sequential stages
            if i < 2:
                x = stages[i](x)

            # Block stages need drop_connect rate
            else:
                for module in stages[i]:
                    drop_connect = drop_connect_rate * block_number / len(self._blocks)
                    block_number += 1.
                    x = module(x, drop_connect)

            features.append(x)
            in_ch = x.shape[1]
            #print(in_ch)
            scse = ChannelSpatialSELayer(in_ch)
            x = scse(x)
            #print(x.shape)
            
          
        return x,features

def _get_pretrained_settings(encoder):
    pretrained_settings = {
        "imagenet": {
            "mean": [0.485, 0.456, 0.406],
            "std": [0.229, 0.224, 0.225],
            "url": url_map[encoder],
            "input_space": "RGB",
            "input_range": [0, 1],
        },
    }
    return pretrained_settings

efficient_net_encoders = {
    "efficientnet-b1": {
        "encoder": EfficientNetEncoder,
        "pretrained_settings": _get_pretrained_settings("efficientnet-b1"),
        "params": {
            "out_channels": (3, 32, 24, 40, 112, 320),
            "stage_idxs": (5, 8, 16, 23),
            "model_name": "efficientnet-b1",
        },
    },
}



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



try:
    from inplace_abn import InPlaceABN
except ImportError:
    InPlaceABN = None

class Conv2dReLU(nn.Sequential):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            padding=0,
            stride=1,
            use_batchnorm=True,
    ):

        if use_batchnorm == "inplace" and InPlaceABN is None:
            raise RuntimeError(
                "In order to use `use_batchnorm='inplace'` inplace_abn package must be installed. "
                + "To install see: https://github.com/mapillary/inplace_abn"
            )

        conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            bias=not (use_batchnorm),
        )
        relu = nn.ReLU(inplace=True)

        if use_batchnorm == "inplace":
            bn = InPlaceABN(out_channels, activation="leaky_relu", activation_param=0.0)
            relu = nn.Identity()

        elif use_batchnorm and use_batchnorm != "inplace":
            bn = nn.BatchNorm2d(out_channels)

        else:
            bn = nn.Identity()

        super(Conv2dReLU, self).__init__(conv, bn, relu)


class Conv2dReLU(nn.Sequential):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            padding=0,
            stride=1,
            use_batchnorm=True,
    ):


        conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            bias=not (use_batchnorm),
        )
        relu = nn.ReLU(inplace=True)

        if use_batchnorm and use_batchnorm != "inplace":
            bn = nn.BatchNorm2d(out_channels)

        else:
            bn = nn.Identity()

        super(Conv2dReLU, self).__init__(conv, bn, relu)



class DecoderBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            skip_channels,
            out_channels,
            use_batchnorm=True,
            attention_type='scse',

    ):
        super().__init__()


        self.conv1 = Conv2dReLU(
            in_channels + skip_channels,
            out_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=use_batchnorm,
        )
        self.attention1 = ChannelSpatialSELayer(num_channels=in_channels + skip_channels)
  
        self.conv2 = Conv2dReLU(
            out_channels,
            out_channels,
            kernel_size=3,
            padding=1,
            use_batchnorm=use_batchnorm,
        )

        self.attention2 = ChannelSpatialSELayer(num_channels=out_channels)


    def forward(self, x, skip=None):

        x = F.interpolate(x, scale_factor=2, mode="nearest")
       
        
        if skip is not None:
           
            x = torch.cat([x, skip], dim=1)
            
            
            x = self.attention1(x)
       
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.attention2(x)
        
        return x

class UnetDecoder(nn.Module):
    def __init__( self):
        super().__init__()

       
        in_channels = [640, 256, 128, 64, 32]
        skip_channels = [112, 40, 24, 32, 0]
        out_channels = [256, 128, 64, 32, 16]




      
        blocks = [
            DecoderBlock(in_ch, skip_ch, out_ch)
            for in_ch, skip_ch, out_ch in zip(in_channels, skip_channels, out_channels)
        ]
        self.blocks = nn.ModuleList(blocks)

    def forward(self,x,features):

       
        features = features[1:]                 # remove first skip with same spatial resolution, i.e, channel feature 1280
       
        features = features[::-1]               # reverse channels to start from head of encoder  features = [640,224,80,48,64]
     

        
        skips = features[1:]                    ## skips = [224,80,48,64]


       

        
        for i, decoder_block in enumerate(self.blocks):
            skip = skips[i] if i < len(skips) else None
            x = decoder_block(x, skip)
           

        return x



In [ ]:
model = UnetDecoder()
print(model)

UnetDecoder(
  (blocks): ModuleList(
    (0): DecoderBlock(
      (conv1): Conv2dReLU(
        (0): Conv2d(752, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (attention1): ChannelSpatialSELayer(
        (cSE): ChannelSELayer(
          (fc1): Linear(in_features=752, out_features=376, bias=True)
          (fc2): Linear(in_features=376, out_features=752, bias=True)
          (relu): ReLU()
          (sigmoid): Sigmoid()
        )
        (sSE): SpatialSELayer(
          (conv): Conv2d(752, 1, kernel_size=(1, 1), stride=(1, 1))
          (sigmoid): Sigmoid()
        )
      )
      (conv2): Conv2dReLU(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    

In [ ]:
import torch.utils.model_zoo as model_zoo

encoders = {}
encoders.update(efficient_net_encoders)

def get_encoder(name, in_channels=3, depth=5, weights=None):

    if weights == None:
      print("Training from Scratch !!")

    try:
        Encoder = encoders[name]["encoder"]
       
    except KeyError:
        raise KeyError("Wrong encoder name `{}`, supported encoders: {}".format(name, list(encoders.keys())))



    params = encoders[name]["params"]
   
    
    params.update(depth=depth)
    
   
    encoder = Encoder(**params)

    if weights is not None:
        try:
            settings = encoders[name]["pretrained_settings"][weights]
        except KeyError:
            raise KeyError("Wrong pretrained weights `{}` for encoder `{}`. Available options are: {}".format(
                weights, name, list(encoders[name]["pretrained_settings"].keys()),
            ))
        encoder.load_state_dict(model_zoo.load_url(settings["url"]))

    encoder.set_in_channels(in_channels)
    

    return encoder

In [ ]:
class SegmentationHead(nn.Sequential):

    def __init__(self, in_channels, out_channels, kernel_size=3, activation='sigmoid', upsampling=1):
        conv2d = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        upsampling = nn.UpsamplingBilinear2d(scale_factor=upsampling) if upsampling > 1 else nn.Identity()
        activation = nn.Sigmoid()
        super().__init__(conv2d, upsampling, activation)



In [ ]:
def initialize_decoder(module):
    for m in module.modules():

        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_uniform_(m.weight, mode="fan_in", nonlinearity="relu")
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

        elif isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)


def initialize_head(module):
    for m in module.modules():
        if isinstance(m, (nn.Linear, nn.Conv2d)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

In [ ]:
model = UnetDecoder()
print(model)

In [ ]:
import argparse
import random
from typing import Union, List
import torch
from torch.autograd import Variable
import numpy as np
import os
import cv2
import sys
sys.argv=['']
del sys


In [ ]:
def parse_args():
    parser = argparse.ArgumentParser(description="Evaluate ISIC dataset.")
    parser.add_argument("--input-data", default="fewshot/fss1000/")
    parser.add_argument(
        "--output-root", help="path to results directory ", default="results/"
    )
    parser.add_argument("-g", "--gpu", type=int, default=0)
    parser.add_argument(
        "-modele",
        "--encoder_model",
        type=str,
        default="b1_model/efficientnet_encoder_99999_1_way_5shot.pkl",
    )
    parser.add_argument(
        "-modeld",
        "--decoder_model",
        type=str,
        default="b1_model/decoder_network_99999_1_way_5shot.pkl",
    )
    parser.add_argument(
        "-modelh",
        "--head_model",
        type=str,
        default="b1_model/segmentation_network_99999_1_way_5shot.pkl",
    )
    parser.add_argument(
        "--use-cuda", type = bool, default = True
    )
    parser.add_argument("--n-support-per-class", type=int, default=5)
    parser.add_argument(
        "--k", default=5, type=int, help="Number of training shots per task"
    )
    args = parser.parse_args()
    return args

In [ ]:
def validate_args(args):
    assert os.path.exists(args.input_data)
    assert os.path.exists(args.encoder_model)
    assert os.path.exists(args.decoder_model)
    assert os.path.exists(args.head_model)


In [ ]:

def build_model(args):
    print("Building neural networks")

    encoder = get_encoder('efficientnet-b1',in_channels=4,weights=None)
    decoder = UnetDecoder()
    head = SegmentationHead(in_channels=16,out_channels=1)

    if args.use_cuda:
        encoder.cuda(args.gpu)
        decoder.cuda(args.gpu)
        head.cuda(args.gpu)

    if os.path.exists(args.encoder_model):
        if args.use_cuda:
            checkpoint1 = torch.load(args.encoder_model)
            encoder.load_state_dict(checkpoint1['encoder_state_dict'])
            print("loaded CUDA-enabled encoder")
  
    else:
        raise RuntimeError(
            "Can not load feature encoder: %s" % args.encoder_model
        )
    if os.path.exists(args.decoder_model):
        if args.use_cuda:
            checkpoint2 = torch.load(args.decoder_model)
            decoder.load_state_dict(checkpoint2['decoder_state_dict'])
            print("loaded GPU decoder")
      
    else:
        raise RuntimeError(
            "Can not load relation network: %s" % args.decoder_model
        )

    if os.path.exists(args.head_model):
        if args.use_cuda:
            checkpoint3 = torch.load(args.head_model)
            head.load_state_dict(checkpoint3['head_state_dict'])
            print("loaded CUDA-enabled head")
        
    else:
        raise RuntimeError(
            "Can not load head: %s" % args.head
        )

    print("Neural networks successfully built.")
    return encoder, decoder, head



In [ ]:
def get_fss_test_set():
    #dirname = os.path.dirname(__file__)
    path = "fss_test_set.txt"  # File containing the test tasks
    
    with open(path, "r") as file:
        tasks = [line.rstrip("\n") for line in file]
    return tasks


In [ ]:
dummy = get_fss_test_set()
print(dummy)


['bus', 'hotel_slipper', 'burj_al', 'reflex_camera', "abe's_flyingfish", 'oiltank_car', 'doormat', 'fish_eagle', 'barber_shaver', 'motorbike', 'feather_clothes', 'wandering_albatross', 'rice_cooker', 'delta_wing', 'fish', 'nintendo_switch', 'bustard', 'diver', 'minicooper', 'cathedrale_paris', 'big_ben', 'combination_lock', 'villa_savoye', 'american_alligator', 'gym_ball', 'andean_condor', 'leggings', 'pyramid_cube', 'jet_aircraft', 'meatloaf', 'reel', 'swan', 'osprey', 'crt_screen', 'microscope', 'rubber_eraser', 'arrow', 'monkey', 'mitten', 'spiderman', 'parthenon', 'bat', 'chess_king', 'sulphur_butterfly', 'quail_egg', 'oriole', 'iron_man', 'wooden_boat', 'anise', 'steering_wheel', 'groenendael', 'dwarf_beans', 'pteropus', 'chalk_brush', 'bloodhound', 'moon', 'english_foxhound', 'boxing_gloves', 'peregine_falcon', 'pyraminx', 'cicada', 'screw', 'shower_curtain', 'tredmill', 'bulb', 'bell_pepper', 'lemur_catta', 'doughnut', 'twin_tower', 'astronaut', 'nintendo_3ds', 'fennel_bulb', 'i

In [ ]:
def get_one_test_shot_batch(data_dir: str, k: int = 5, class_num: int = 1):
    test_shots: int = 1

      
    support_images = np.zeros((class_num * k, 3, 224, 224), dtype=np.float32)
    support_labels = np.zeros((class_num * k, class_num, 224, 224), dtype=np.float32)
    query_images = np.zeros((class_num * test_shots, 3, 224, 224), dtype=np.float32)
    query_labels = np.zeros(
        (class_num * test_shots, class_num, 224, 224), dtype=np.float32
    )
    zeros = np.zeros((class_num * test_shots, 1, 224, 224), dtype=np.float32)

    # Load tuples of images and masks in data_dir
    labels = [x for x in os.listdir(data_dir) if x.endswith(".png")]
    images = [x.replace(".png", ".jpg") for x in labels]
    labels = [os.path.join(data_dir, x) for x in labels]
    images = [os.path.join(data_dir, x) for x in images]
    images_labels = list(zip(images, labels))

    # Randomly sample k support tuples
    random.shuffle(images_labels)
    images_labels = [
        (
            
            np.transpose(cv2.resize(cv2.imread(i),(224,224))[:, :, ::-1] / 255.0, (2, 0, 1)),
            cv2.resize(cv2.imread(l),(224,224))[:, :, 0],
        )
        for i, l in images_labels
    ]
    images_labels = [
        (i, l)
        for i, l in images_labels
        if i.shape[1] == 224
        and i.shape[2] == 224
        and l.shape[0] == 224
        and l.shape[1] == 224
    ]
    #print(len(images_labels))
    assert k <= len(images_labels) - test_shots

    support_image_labels = images_labels[:k]
    test_image_labels = images_labels[k : k + test_shots]

    for i, (image, label) in enumerate(support_image_labels):
        support_images[i] = image
        support_labels[i][0] = label
    support_images_tensor = torch.from_numpy(support_images)
    support_labels_tensor = torch.from_numpy(support_labels)
    support_images_tensor = torch.cat(
        (support_images_tensor, support_labels_tensor), dim=1
    )

    for i, (image, label) in enumerate(test_image_labels):
        query_images[i] = image
        query_labels[i][0] = label
    zeros_tensor = torch.from_numpy(zeros)
    query_images_tensor = torch.from_numpy(query_images)
    query_images_tensor = torch.cat((query_images_tensor, zeros_tensor), dim=1)
    query_labels_tensor = torch.from_numpy(query_labels)

    return (
        support_images_tensor,
        support_labels_tensor,
        query_images_tensor,
        query_labels_tensor,
    )


In [ ]:
def forward(
    encoder, decoder, head, support_images_tensor, query_images_tensor, args
):
    class_num = 1
    if args.use_cuda:
        var = Variable(support_images_tensor).cuda(args.gpu)
    else:
        var = Variable(support_images_tensor)
    sample_features, _ = encoder(var)
    sample_features = sample_features.view(class_num, args.k, 320, 7, 7)
    sample_features = torch.sum(sample_features, 1).squeeze(1)  # 1*512*7*7
    if args.use_cuda:
        batch_features, ft_list = encoder(
            Variable(query_images_tensor).cuda(args.gpu)
        )
    else:
        batch_features, ft_list = encoder(Variable(query_images_tensor))
        
    sample_features_ext = sample_features.unsqueeze(0).repeat(class_num, 1, 1, 1, 1)
    batch_features_ext = batch_features.unsqueeze(0).repeat(class_num, 1, 1, 1, 1)
    batch_features_ext = torch.transpose(batch_features_ext, 0, 1)
    relation_pairs = torch.cat((sample_features_ext, batch_features_ext), 2).view(
        -1, 640, 7, 7
    )
    out = decoder(relation_pairs,ft_list)
    output = head(out).view(-1,class_num,224,224)
    return output


In [ ]:
def evaluate_predictions(predictions, labels, iou_as_percentage = False):
    pred = predictions.data.cpu().numpy()
    ious = []
    pred[pred <= 0.5] = 0
    pred[pred > 0.5] = 1
    testlabel = labels.numpy()[0][0].astype(bool)
    pred = pred.astype(bool)
    # Compute IOU
    overlap = testlabel * pred
    union = testlabel + pred
    iou = overlap.sum() / float(union.sum())
    print("iou=%0.4f" % iou)
    if iou_as_percentage:
        iou *= 100
    #print("iou=%0.2f" % iou)
    ious.append(iou)
    return ious



In [ ]:
def ci95(a: Union[List[float], np.ndarray]):
    """Computes the 95% confidence interval of the array `a`."""
    sigma = np.std(a)
    return 1.96 * sigma / np.sqrt(len(a))

In [ ]:
print(support_images_tensor)

In [ ]:
def main():
    args = parse_args()
    validate_args(args)
    n_eval_samples_per_task = 2  # Following Hendryx et al. 2020

    # Get the 240 test tasks:
    test_tasks = get_fss_test_set()
    test_tasks = [os.path.join(args.input_data, x) for x in test_tasks]

    

    encoder, decoder, head = build_model(args)

    # Loop through each task, evaluating each task n_eval_samples_per_task times
    ious = []
    for _ in range(n_eval_samples_per_task):
        for task in test_tasks:
            # Load examples for the task:
            (
                support_images_tensor,
                support_labels_tensor,
                query_images_tensor,
                query_labels_tensor,
            ) = get_one_test_shot_batch(task, k=args.k)

            predictions = forward(
                encoder,
                decoder,
                head,
                support_images_tensor,
                query_images_tensor,
                args,
            )

            # Evaluate the IoU on queries:
            ious.extend(evaluate_predictions(predictions, query_labels_tensor))

    # Report mean and 95% CI:
    print(f"Mean IoU +- 95% CI: {np.nanmean(ious)} +- {ci95(ious)}")
    print(f"Mean IoU: {np.nanmean(ious)*100:.2f} +/- {ci95(ious):.2f}")


In [ ]:
if __name__ == "__main__":
    main()

Building neural networks
Training from Scratch !!
loaded CUDA-enabled encoder
loaded GPU decoder
loaded CUDA-enabled head
Neural networks successfully built.
iou=0.8957
iou=0.8961
iou=0.8885
iou=0.4315
iou=0.9079
iou=0.7893
iou=0.5706
iou=0.6259
iou=0.6732
iou=0.5902
iou=0.9519
iou=0.8562
iou=0.6959
iou=0.9417
iou=0.7509
iou=0.9637
iou=0.3265
iou=0.8260
iou=0.9216
iou=0.7691
iou=0.9380
iou=0.7894
iou=0.8757
iou=0.9397
iou=0.5104
iou=0.7302
iou=0.9775
iou=0.9443
iou=0.9444
iou=0.7950
iou=0.7899
iou=0.9198
iou=0.9260
iou=0.3999
iou=0.2505
iou=0.5043
iou=0.1785
iou=0.8937
iou=0.9394
iou=0.8074
iou=0.8746
iou=0.9391
iou=0.6133
iou=0.8964
iou=0.9064
iou=0.9013
iou=0.8270
iou=0.8293
iou=0.8556
iou=0.6194
iou=0.9088
iou=0.5465
iou=0.8937
iou=0.9823
iou=0.8824
iou=0.4554
iou=0.9267
iou=0.9731
iou=0.8655
iou=0.9585
iou=0.3035
iou=0.3943
iou=0.6846
iou=0.8248
iou=0.9316
iou=0.9711
iou=0.8835
iou=0.9794
iou=0.5597
iou=0.9461
iou=0.6427
iou=0.8987
iou=0.8290
iou=0.6732
iou=0.7602
iou=0.8649
iou=0.